# Google AlloyDB for PostgreSQL

> [AlloyDB](https://cloud.google.com/alloydb) 是一项全托管的关系型数据库服务，提供高**性能**、无缝**集成**和出色的**可扩展性**。AlloyDB 与 PostgreSQL 完全兼容。利用 AlloyDB 的 Langchain 集成，扩展您的数据库应用程序，构建支持 AI 的体验。

本 Notebook 将介绍如何使用 `AlloyDBLoader` 类通过 `AlloyDB for PostgreSQL` 加载 Documents。

在 [GitHub](https://github.com/googleapis/langchain-google-alloydb-pg-python/) 上了解有关该软件包的更多信息。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-alloydb-pg-python/blob/main/docs/document_loader.ipynb)

## 开始之前

要运行此笔记本，您需要执行以下操作：

*   [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
*   [启用 AlloyDB API](https://console.cloud.google.com/flows/enableapi?apiid=alloydb.googleapis.com)
*   [创建 AlloyDB 集群和实例](https://cloud.google.com/alloydb/docs/cluster-create)
*   [创建 AlloyDB 数据库](https://cloud.google.com/alloydb/docs/quickstart/create-and-connect)
*   [向数据库添加用户](https://cloud.google.com/alloydb/docs/database-users/about)

### 🦜🔗 库安装
安装集成库 `langchain-google-alloydb-pg`。

In [ ]:
%pip install --upgrade --quiet  langchain-google-alloydb-pg

**仅限 Colab：** 取消注释以下单元格以重启内核，或使用按钮重启内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 🔐 身份验证
使用登录到此笔记本的 IAM 用户进行 Google Cloud 身份验证，以便访问您的 Google Cloud 项目。

* 如果您正在使用 Colab 运行此笔记本，请使用下面的单元格并继续。
* 如果您正在使用 Vertex AI Workbench，请查看[此处](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)的设置说明。

In [ ]:
from google.colab import auth

auth.authenticate_user()

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便您可以在此 notebook 中利用 Google Cloud 资源。

如果您不知道您的项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @title Project { display-mode: "form" }
PROJECT_ID = "gcp_project_id"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

## 基本用法

### 设置 AlloyDB 数据库变量
在 [AlloyDB 实例页面](https://console.cloud.google.com/alloydb/clusters) 中查找您的数据库值。

In [4]:
# @title Set Your Values Here { display-mode: "form" }
REGION = "us-central1"  # @param {type: "string"}
CLUSTER = "my-cluster"  # @param {type: "string"}
INSTANCE = "my-primary"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
TABLE_NAME = "vector_store"  # @param {type: "string"}

### AlloyDBEngine 连接池

将 AlloyDB 作为向量存储的要求之一是需要一个 `AlloyDBEngine` 对象。`AlloyDBEngine` 会配置一个到你的 AlloyDB 数据库的连接池，从而支持你的应用程序成功建立连接，并遵循行业最佳实践。

要使用 `AlloyDBEngine.from_instance()` 创建一个 `AlloyDBEngine`，你只需要提供 5 项信息：

1. `project_id`：AlloyDB 实例所在的 Google Cloud 项目的项目 ID。
1. `region`：AlloyDB 实例所在的区域。
1. `cluster`：AlloyDB 集群的名称。
1. `instance`：AlloyDB 实例的名称。
1. `database`：要连接到 AlloyDB 实例上的数据库的名称。

默认情况下，将使用 [IAM 数据库身份验证](https://cloud.google.com/alloydb/docs/connect-iam) 作为数据库身份验证方法。此库使用来自环境的 [应用程序默认凭据 (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials) 所属的 IAM 主体。

你也可以选择使用 [内置数据库身份验证](https://cloud.google.com/alloydb/docs/database-users/about)，即使用用户名和密码来访问 AlloyDB 数据库。只需向 `AlloyDBEngine.from_instance()` 提供可选的 `user` 和 `password` 参数即可：

* `user`：用于内置数据库身份验证和登录的数据库用户
* `password`：用于内置数据库身份验证和登录的数据库密码。

**注意**: 本教程演示了异步接口。所有异步方法都有对应的同步方法。

In [ ]:
from langchain_google_alloydb_pg import AlloyDBEngine

engine = await AlloyDBEngine.afrom_instance(
    project_id=PROJECT_ID,
    region=REGION,
    cluster=CLUSTER,
    instance=INSTANCE,
    database=DATABASE,
)

### 创建 AlloyDBLoader

In [ ]:
from langchain_google_alloydb_pg import AlloyDBLoader

# Creating a basic AlloyDBLoader object
loader = await AlloyDBLoader.create(engine, table_name=TABLE_NAME)

### 通过默认表加载文档
加载器使用第一列作为 `page_content`，所有其他列作为元数据，从表中返回文档列表。默认表的第一列将是 `page_content`，第二列将是元数据（JSON）。每一行将成为一个文档。

In [ ]:
docs = await loader.aload()
print(docs)

### 通过自定义表/元数据或自定义页面内容列加载文档

In [ ]:
loader = await AlloyDBLoader.create(
    engine,
    table_name=TABLE_NAME,
    content_columns=["product_name"],  # Optional
    metadata_columns=["id"],  # Optional
)
docs = await loader.aload()
print(docs)

### 设置页面内容格式
加载器返回一个 Documents 列表，每行一个 document，页面内容以指定的字符串格式返回，例如 text（空格分隔的连接字符串）、JSON、YAML、CSV 等。JSON 和 YAML 格式包含头部信息，而 text 和 CSV 则不包含字段头部信息。

In [ ]:
loader = AlloyDBLoader.create(
    engine,
    table_name="products",
    content_columns=["product_name", "description"],
    format="YAML",
)
docs = await loader.aload()
print(docs)